In [22]:
import pandas as pd
import plotly.express as px
from geopy.geocoders import Nominatim
import time

## Find latitudes and longitudes for each address

In [23]:
# Run this code only if you add new galleries/addresses to extract lat and lon (takes about 1 minute)
df = pd.read_excel("galleries.xlsx", sheet_name="Manhattan")

# Initialize geolocator
geolocator = Nominatim(user_agent="gallery_mapper")

# Function to get latitude and longitude
def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        time.sleep(1)  # To avoid rate limiting
        if location:
            return pd.Series([location.latitude, location.longitude])
    except:
        return pd.Series([None, None])
    return pd.Series([None, None])

# Geocode addresses if coordinates not present
df[['Latitude', 'Longitude']] = df['Address'].apply(geocode_address)

# Drop rows with missing coordinates
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Create hover text
df['HoverText'] = df.apply(
    lambda row: f"<b>{row['Gallery Name']}</b><br>{row['Address']}<br><a href='{row['Website']}'>{row['Website']}</a>",
    axis=1
)

df.to_excel("galleries_manhattan_lat_lon.xlsx", index=False)



WARNING	Task(Task-2) urllib3.connectionpool:connectionpool.py:urlopen()- Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=555+W+24th+St%2C+New+York%2C+NY+10011&format=json&limit=1
WARNING	Task(Task-2) urllib3.connectionpool:connectionpool.py:urlopen()- Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=555+W+24th+St%2C+New+York%2C+NY+10011&format=json&limit=1
WARNING	Task(Task-2) urllib3.connectionpool:connectionpool.py:urlopen()- Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=4

## Create the map

In [27]:
# Read the Excel files with lat and lon
df_queens = pd.read_excel("galleries_queens_lat_lon.xlsx")
df_bronx = pd.read_excel("galleries_bronx_lat_lon.xlsx")
df_brooklyn = pd.read_excel("galleries_brooklyn_lat_lon.xlsx")
df_manhattan = pd.read_excel("galleries_manhattan_lat_lon.xlsx")

# Add the 'Borough' column
df_queens['Borough'] = 'Queens'
df_bronx['Borough'] = 'Bronx'
df_brooklyn['Borough'] = 'Brooklyn'
df_manhattan['Borough'] = 'Manhattan'

# Concatenate vertically
df_combined = pd.concat([df_queens, df_bronx, df_brooklyn, df_manhattan], ignore_index=True)

# Preview the result
print(df_combined.tail(2))

             Gallery Name                            Address  \
125         Albertz Benda  515 W 26th St, New York, NY 10001   
126  Simone Subal Gallery     131 Bowery, New York, NY 10002   

           Email / Contact                   Website   Latitude  Longitude  \
125  info@albertzbenda.com  https://albertzbenda.com  40.750147 -74.003524   
126   info@simonesubal.com   https://simonesubal.com  40.718550 -73.994448   

                                             HoverText    Borough  
125  <b>Albertz Benda</b><br>515 W 26th St, New Yor...  Manhattan  
126  <b>Simone Subal Gallery</b><br>131 Bowery, New...  Manhattan  


In [28]:
fig = px.scatter_map(
    df_combined,
    lat="Latitude",
    lon="Longitude",
    hover_name="Gallery Name",
    hover_data={"Latitude": False, "Longitude": False, "HoverText": True},
    text="Gallery Name",
    custom_data=["HoverText"],
    color="Borough",  # Use the Borough column for color
    color_discrete_map={
        "Queens": "#272827",
        "Bronx": "#ea2081",
        "Brooklyn": "#52bcff",
        "Manhattan": "#EEA705"
    },
    zoom=10,
    height=800
)

fig.update_traces(
    marker=dict(size=8, symbol="circle"),
    hovertemplate="%{customdata[0]}"
)

fig.update_layout(
    mapbox_style="open-street-map",
    title="Art Galleries New York City",
    margin={"r":0,"t":40,"l":0,"b":0},
    hoverlabel=dict(
        bgcolor="#f0f0f0", 
        font=dict(color="#272827")
    )
)

fig.write_html("galleries_map.html")
fig.show()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geopy]
Note: you may need to restart the kernel to use updated packages.
